<a href="https://colab.research.google.com/github/lsteffenel/CHPS0906/blob/main/TP1/6-Visualiser%20un%20pooling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Couche de pooling max

Dans ce notebook nous ajoutons et visualisons la sortie d'une couche de pooling max dans un CNN.

Une couche convolutionnelle + fonction d'activation, suivie d'une couche de pooling et d'une couche linéaire (pour créer une taille de sortie souhaitée) constituent les couches de base d'un CNN.

<img src='https://github.com/udacity/deep-learning-v2-pytorch/blob/master/convolutional-neural-networks/conv-visualization/notebook_ims/CNN_all_layers.png?raw=1' height=50% width=50% />

### Importer une image

In [ ]:
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

# TODO: Déposer une image dans l'onglet "Fichiers". Ensuite, renseignez le nom du fichier ci-dessous
img_path = 'car_sdc.png'

# load color image
bgr_img = cv2.imread(img_path)
# convert to grayscale
gray_img = cv2.cvtColor(bgr_img, cv2.COLOR_BGR2GRAY)

# normalize, rescale entries to lie in [0,1]
gray_img = gray_img.astype("float32")/255

# plot image
plt.imshow(gray_img, cmap='gray')
plt.show()

### Définir et visualiser les filtres

In [ ]:
import numpy as np

## TODO: Feel free to modify the numbers here, to try out another filter!
filter_vals = np.array([[-1, -1, 1, 1], [-1, -1, 1, 1], [-1, -1, 1, 1], [-1, -1, 1, 1]])

print('Filter shape: ', filter_vals.shape)


In [ ]:
# Defining four different filters,
# all of which are linear combinations of the `filter_vals` defined above

# define four filters
filter_1 = filter_vals
filter_2 = -filter_1
filter_3 = filter_1.T
filter_4 = -filter_3
filters = np.array([filter_1, filter_2, filter_3, filter_4])

# For an example, print out the values of filter 1
print('Filter 1: \n', filter_1)

### Définir les couches convolutionnelles et de pooling

Vous avez déjà vu comment définir une couche convolutionnelle. L'étape suivante est celle de définir une **Couche de pooling**.

Dans le paragraphe suivant, nous initialisons une couche convolutionnelle afin qu'elle contienne tous les filtres créés. Ajoutez ensuite une couche de pooling max, [documentée ici](http://pytorch.org/docs/stable/_modules/torch/nn/modules/pooling.html), avec une taille de filtre de (2x2) afin que vous puissiez voir que la résolution de l'image a été réduite après cette étape !

Une couche de pooling max réduit la taille x-y d'une entrée et ne conserve que les valeurs de pixels les plus *actives*. Vous trouverez ci-dessous un exemple de noyau de pooling 2x2, avec un pas de 2, appliqué à un petit patch de valeurs de pixels en niveaux de gris ; réduisant la taille x-y du patch d'un facteur 2. Seules les valeurs de pixels maximales dans 2x2 restent dans la nouvelle sortie groupée.

<img src='https://github.com/udacity/deep-learning-v2-pytorch/blob/master/convolutional-neural-networks/conv-visualization/notebook_ims/maxpooling_ex.png?raw=1' hauteur=50% largeur=50% />

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# define a neural network with a convolutional layer with four filters
# AND a pooling layer of size (2, 2)
class Net(nn.Module):

    def __init__(self, weight):
        super(Net, self).__init__()
        # initializes the weights of the convolutional layer to be the weights of the 4 defined filters
        k_height, k_width = weight.shape[2:]
        # assumes there are 4 grayscale filters
        self.conv = nn.Conv2d(1, 4, kernel_size=(k_height, k_width), bias=False)
        self.conv.weight = torch.nn.Parameter(weight)
        # define a pooling layer
        self.pool = nn.MaxPool2d(2, 2)

    def forward(self, x):
        # calculates the output of a convolutional layer
        # pre- and post-activation
        conv_x = self.conv(x)
        activated_x = F.relu(conv_x)

        # applies pooling layer
        pooled_x = self.pool(activated_x)

        # returns all layers
        return conv_x, activated_x, pooled_x

# instantiate the model and set the weights
weight = torch.from_numpy(filters).unsqueeze(1).type(torch.FloatTensor)
model = Net(weight)

# print out the layer in the network
print(model)

### Visualisez la sortie de chaque filtre

Nous allons d'abord définir une fonction d'assistance, `viz_layer`, qui prend en compte une couche spécifique et un nombre de filtres (argument facultatif) et affiche la sortie de cette couche une fois qu'une image a été transmise.

In [ ]:
# helper function for visualizing the output of a given layer
# default number of filters is 4
def viz_layer(layer, n_filters= 4):
    fig = plt.figure(figsize=(20, 20))

    for i in range(n_filters):
        ax = fig.add_subplot(1, n_filters, i+1)
        # grab layer outputs
        ax.imshow(np.squeeze(layer[0,i].data.numpy()), cmap='gray')
        ax.set_title('Output %s' % str(i+1))

Regardons la sortie d'une couche convolutionnelle après l'application d'une fonction d'activation ReLu.

#### Activation ReLu

Une fonction ReLu transforme toutes les valeurs de pixels négatives en 0 (noir). Voir l'équation illustrée ci-dessous pour les valeurs de pixels d'entrée, `x`.

<img src='https://github.com/udacity/deep-learning-v2-pytorch/blob/master/convolutional-neural-networks/conv-visualization/notebook_ims/relu_ex.png?raw=1' height=50% width=50% />

In [ ]:
# plot original image
plt.imshow(gray_img, cmap='gray')

# visualize all filters
fig = plt.figure(figsize=(12, 6))
fig.subplots_adjust(left=0, right=1.5, bottom=0.8, top=1, hspace=0.05, wspace=0.05)
for i in range(4):
    ax = fig.add_subplot(1, 4, i+1, xticks=[], yticks=[])
    ax.imshow(filters[i], cmap='gray')
    ax.set_title('Filter %s' % str(i+1))


# convert the image into an input Tensor
gray_img_tensor = torch.from_numpy(gray_img).unsqueeze(0).unsqueeze(1)

# get all the layers
conv_layer, activated_layer, pooled_layer = model(gray_img_tensor)

# visualize the output of the activated conv layer
viz_layer(activated_layer)

### Visualisez la sortie de la couche de pooling

Ensuite, regardez la sortie d'une couche de pooling. La couche pooling prend en entrée les cartes de caractéristiques illustrées ci-dessus et réduit la dimensionnalité de ces cartes par un facteur de compression, en construisant une nouvelle image plus petite contenant uniquement les valeurs maximales (les plus lumineuses) dans une zone de noyau donnée.

Regardez les valeurs sur les axes x, y pour voir comment l'image a changé de taille.

In [ ]:
# visualize the output of the pooling layer
viz_layer(pooled_layer)